# Linear Regression

In this tutorial we will implement a linear regression model. We will also implement a function that splits the available data into a training and a testing part.

## Problem Setting

We will use the Boston Housing Dataset. This dataset contains information collected by the U.S Census Service concerning housing in the city of Boston in the state of Massachusetts in 1978. Our goal is to predict the median value of the houses in a particular town in the city of Boston given its attributes. Check the file ’housing_features_description.txt’ for more information on the attributes.

In [1]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### NOTE: The boston dataset has an ethical problem. More on this can e.g. be found in the scikit documentation. ###
### Summary: The dataset contains the proportion of black people, which was assumed that racial self-segregation had a positive impact on house prices. ###


data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
boston_data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
boston_target = raw_df.values[1::2, 2]

df=pd.DataFrame(boston_data)
df.columns=['crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct']
X=boston_data
y=boston_target


In [2]:
df.head(10)

,crime_rate,res_land_zoned,industry,charles_river,nox,avg_num_rooms,prop_bf_1940,dst_emply_center,rd_highway_idx,tax_rate,stdnt_tchr_ratio,prop_blacks,low_status_pct
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


# Note 1:

Think about the ethical aspects of this dataset and machine learning in general. 

Can you always trust your data source? Can we use every possible information for our models?

### Exercise 1

Write the *split_train_test(X,y,split,seed)*, given an instance matrix $X \in \mathbb{R}^{N \times D}$, labels $y \in Y^N$, a split ratio in $[0, 1]$ and a random seed $\in \mathbb{Z}$. Split the dataset in $(split×100)\%$ of the instances for training our model and the rest for testing, i.e. 

$$ \left|X_{\text{train}}\right| = \lceil \text{split} \cdot N \rceil, \qquad |X_{\text{train}}| + |X_{\text{test}}| = N. $$
Make sure you use the given random number generator seed so we all get the same results. The function is supposed to return:

- X_train, y_train: the training instances and labels;
- X_test, y_test: the test instances and labels,

in the same order as was mentioned.

Hint: It may be helpful to use shuffling functionality (e.g. np.random.shuffle).

In [14]:
def split_train_test(X, y, split, seed):
    # return X_train, X_test, y_train, y_test = train_test_split(X_versi, y_versi, test_size=split, random_state=seed)
    # Set the random seed for reproducibility
    np.random.seed(seed)
    
    # Number of instances
    N = X.shape[0]
    
    # Generate a shuffled index array
    indices = np.arange(N)
    np.random.shuffle(indices)
    
    # Calculate the split index
    split_index = int(np.ceil(split * N))
    
    # Split the indices into training and testing
    train_indices = indices[:split_index]
    test_indices = indices[split_index:]
    
    # Split the data into training and testing sets
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    return X_train, y_train, X_test, y_test



### Exercise 2

Write the function *train_linear_reg(X_train,y_train,lmbd)*.
Implement the ridge regression model (slide 24). The function should output the learned weight vector $\theta \in \mathbb{R}^D$ or $\mathbb{R}^{D+1}$ (depending on whether you are adding *bias*).

In [15]:
def train_linear_reg(X_train, y_train, lmbd):
    # Add bias term (column of ones) to X_train
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    
    # Compute the regularization matrix (lambda * I), excluding the bias term from regularization
    I = np.eye(X_train.shape[1])
    I[0, 0] = 0  # Do not regularize the bias term
    
    # Compute X^T * X
    XtX = X_train.T @ X_train
    
    # Compute the inverse of (X^T * X + lambda * I)
    XtX_lmbd_I_inv = np.linalg.inv(XtX + lmbd * I)
    
    # Compute X^T * y
    Xty = X_train.T @ y_train
    
    # Compute the weight vector theta
    theta = XtX_lmbd_I_inv @ Xty
    
    return theta


### Exercise 3

Write the function *predict(X,theta)* which predicts housing values vector pred for a dataset X and a previously trained parameter vector $\theta$.

In [16]:
def predict(X, theta):
    # Add bias term (column of ones) to X
    X_augmented = np.hstack((np.ones((X.shape[0], 1)), X))
    
    # Compute the predictions
    y_pred = X_augmented @ theta
    
    return y_pred


### Exercise 4

Write the function *mean_abs_loss(y_true,y_pred)* which computes the mean of the absolute differences between our prediction vector $y\_pred$ and the real housing values $y\_true$.

In [17]:
def mean_abs_loss(y_true, y_pred):
    # Compute the absolute differences
    absolute_differences = np.abs(y_true - y_pred)
    
    # Compute the mean of the absolute differences
    mean_absolute_loss = np.mean(absolute_differences)
    
    return mean_absolute_loss

### Exercise 5

Evaluate your solutions by running the following code. 

Moreover, answer the following questions: What is the most important feature in your model? Are there features that are not so important? What happens if you remove them? Are there outliers with a high absolute loss?

In [19]:
seed = 3
lmbd=1
split=0.7
X_train,y_train,X_test,y_test=split_train_test(X,y,split,seed)
theta=train_linear_reg(X_train,y_train,lmbd)
y_pred=predict(X_test,theta)
mae=mean_abs_loss(y_test,y_pred)
print ('The mean absolute loss is {loss:0.3f}'.format(loss=mae*1000))

The mean absolute loss is 3428.558


In [22]:

# Calculate mean absolute loss
loss = mean_abs_loss(y_test, y_pred)
print("Mean Absolute Loss:", loss)

# Analyze feature importance
feature_importance = pd.DataFrame({
    'Feature': ['Bias','crime_rate','res_land_zoned','industry','charles_river','nox','avg_num_rooms','prop_bf_1940','dst_emply_center','rd_highway_idx','tax_rate','stdnt_tchr_ratio','prop_blacks','low_status_pct'],
    'Coefficient': theta
})
print(feature_importance)

# Identify outliers with high absolute loss
absolute_differences = np.abs(y_test - y_pred)
outliers = pd.DataFrame({
    'True Value': y_test,
    'Predicted Value': y_pred,
    'Absolute Loss': absolute_differences
})
outliers = outliers[outliers['Absolute Loss'] > 1]  # Threshold for high absolute loss
print("Outliers with High Absolute Loss:")
print(outliers)


Mean Absolute Loss: 3.428557739962404
             Feature  Coefficient
0               Bias    32.535187
1         crime_rate    -0.034069
2     res_land_zoned     0.041596
3           industry     0.017369
4      charles_river     2.562492
5                nox   -10.193440
6      avg_num_rooms     3.832525
7       prop_bf_1940    -0.009998
8   dst_emply_center    -1.447981
9     rd_highway_idx     0.281418
10          tax_rate    -0.015609
11  stdnt_tchr_ratio    -0.869075
12       prop_blacks     0.011214
13    low_status_pct    -0.576112
Outliers with High Absolute Loss:
     True Value  Predicted Value  Absolute Loss
1           7.4         5.837220       1.562780
2          19.0        13.020089       5.979911
4          33.2        35.486706       2.286706
5          16.5        10.052798       6.447202
6          20.3        23.867555       3.567555
..          ...              ...            ...
145        12.7        11.678760       1.021240
146        10.5        13.681365  